In [1]:
import os
import pickle
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import CrossEncoder

from utils.llmclass import LLM
from utils.search import HybridSearch
# Configuration
DB_PATH = os.path.join(os.getcwd(), "seamanuals")  # Safe path joining
BM25_PATH = os.path.join(DB_PATH, "bm25_retriever.pkl")

In [2]:
client = chromadb.PersistentClient(path=DB_PATH)

embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="BAAI/bge-base-en-v1.5")
collection = client.get_or_create_collection(name="Sea-Database",
                                             embedding_function=embedding_func)
with open(BM25_PATH, "rb") as f:
    bm25_retriever = pickle.load(f)

In [3]:
search = HybridSearch.get(collection, bm25_retriever)

In [4]:
query = "According to the Boat Crew Training Handbook glossary, how is the term \u201cKnot (kn or kt)\u201d defined, and what specific measurements and additional meanings does it encompass?"
docs = search(query)

print("--- Retrieved Context ---")
for doc in docs:
    print(doc['rank'], doc['source'], doc['content'])
    # print(f"Content: \n{doc['content']}")

--- Retrieved Context ---
1 BOAT CREW HANDBOOK - 16114.1B_Boat Operations.pdf Boat Crew Training Handbook – Boat Operations
Appendix A – Glossary
TERM DEFINITION
Dolphin A structure consisting of a number of piles driven into the seabed or river bed in a circular
pattern and drawn together with wire rope. May be used as part of a dock structure or a
minor aid to navigation. Commonly used when a single pile would not provide the desired
strength.
Draft The point on a vessel’s underwater body, measured from the waterline, that reaches the
greatest depth.
Drag Forces opposing direction of motion due to friction, profile and other components. The
amount that a ship is down by the stern.
Drift The rate/speed at which a vessel moves due to the effects of wind, wave, current, or the
accumulative effects of each. Usually expressed in knots.
2 BOAT CREW HANDBOOK - 16114.1B_Boat Operations.pdf Boat Crew Training Handbook – Boat Operations
APPENDIX A Glossary
Introduction This appendix contains a

In [5]:
rag = LLM(model_name='qwen-3-32b')

In [6]:
# 1. SIMULATE RETRIEVAL
query = "According to the Boat Crew Training Handbook glossary, how is the term \u201cKnot (kn or kt)\u201d defined, and what specific measurements and additional meanings does it encompass?"
docs = search(query)
context = ["-> ".join([doc['source'], doc['content']]) for doc in docs]
# 2. GENERATE RESPONSE
print(f"Query: {query}\n")
answer = rag.generate_answer(query, context)
print(f"Response:\n{answer}")

Query: What is the procedure when battling fires abroad other boats?

Response:
<think>
Okay, the user is asking about the procedure for battling fires on other boats. Let me check the provided context.

Looking through the context, there's a section titled "H.7. Fires Aboard Other Boats" with a step-by-step procedure. The steps include briefing crew members, assigning duties, establishing communication, approaching upwind, checking for people in the water, advising those on the boat to move to a safe area, determining the fire's extent, evacuating if necessary, checking survivors' conditions, and fighting the fire if it's manageable. There's also mention of when a fire is under control.

I need to make sure I only use the information from the context provided. The user wants the answer based solely on the given documents. The steps listed in H.7 are the key here. Also, there's a table with steps 1 through 11 which outline the procedures. I should present these steps in order, paraphra